<a href="https://colab.research.google.com/github/sistelca/NOT-MNIST/blob/master/2_fullyconnected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# corre solo para descomprimir la primera vez

from zipfile import ZipFile

filename = "/content/gdrive/My Drive/curso_udacity/notMNIST_nube.zip"

	
with ZipFile(filename) as zipObj:
  # Extract all the contents of zip file in current directory
  zipObj.extractall("/content/gdrive/My Drive/curso_udacity/")


In [2]:
pickle_file = '/content/gdrive/My Drive/curso_udacity//notMNIST_nube.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


Training set (10000, 784) (10000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset)
  tf_train_labels = tf.constant(train_labels)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.707222
Training accuracy: 9.3%
Validation accuracy: 10.3%
Loss at step 100: 2.276988
Training accuracy: 72.6%
Validation accuracy: 71.3%
Loss at step 200: 1.823069
Training accuracy: 75.1%
Validation accuracy: 74.1%
Loss at step 300: 1.584183
Training accuracy: 76.3%
Validation accuracy: 74.9%
Loss at step 400: 1.425965
Training accuracy: 77.4%
Validation accuracy: 75.3%
Loss at step 500: 1.308938
Training accuracy: 78.0%
Validation accuracy: 75.6%
Loss at step 600: 1.216931
Training accuracy: 78.6%
Validation accuracy: 75.8%
Loss at step 700: 1.141590
Training accuracy: 79.1%
Validation accuracy: 75.9%
Loss at step 800: 1.078250
Training accuracy: 79.7%
Validation accuracy: 76.2%
Test accuracy: 83.8%


Pasemos ahora al entrenamientoSGD, que es mucho más rápido.

El gráfo será similar, excepto que en lugar de mantener todos los datos de entrenamiento en un nodo constante, creamos un nodo marcador de posición que será alimentado con datos reales en cada llamada de session.run().

In [0]:

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.587482
Minibatch accuracy: 7.8%
Validation accuracy: 12.6%
Minibatch loss at step 500: 1.561721
Minibatch accuracy: 77.3%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.195530
Minibatch accuracy: 81.2%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 0.680323
Minibatch accuracy: 82.0%
Validation accuracy: 76.2%
Minibatch loss at step 2000: 0.947160
Minibatch accuracy: 79.7%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 0.762678
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.652506
Minibatch accuracy: 84.4%
Validation accuracy: 77.1%
Test accuracy: 84.5%


# Problema

Convierta el ejemplo de regresión logística con SGD en una red neuronal de una capa oculta con unidades lineales rectificadas nn.relu() y 1024 nodos ocultos. Este modelo debería mejorar su validación / precisión de la prueba

In [5]:
batch_size = 128
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # Capa 1
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # Capa 2 de salida
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))
    
    def NN(x): # neurona
        """
            x: matriz
               su forma debe ser (m, 784)

            regresa la activacion de la capa de salida
            matriz de (m, 10)
        """

        # Capa oculta 1
        z_1 = tf.matmul(x,w_1) + b_1 # lineal
        a_1 = tf.nn.relu(z_1)	# activacion

        # Capa 2 salida
        z_2 = tf.matmul(a_1,w_2) + b_2

        return z_2
    
    # Training computation.
    
    y_ = NN(tf_train_dataset)
    
    # funcion de costo
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=y_))
    
    # Optimizador.
    optim = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(y_)
    valid_prediction = tf.nn.softmax(NN(tf_valid_dataset))
    test_prediction = tf.nn.softmax(NN(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optim, loss, y_], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test presicion Global: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 385.027832
Minibatch accuracy: 5.5%
Validation accuracy: 27.0%
Minibatch loss at step 500: 0.921902
Minibatch accuracy: 97.7%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.385438
Minibatch accuracy: 98.4%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 0.000308
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 82.9%
Minibatch loss at step 2500: 0.000044
Minibatch accuracy: 100.0%
Validation accuracy: 82.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 82.9%
Test presicion Global: 89.8%
